In [1]:
# loading libraries
import pandas as pd
import networkx as nx
import os
from tqdm import tqdm
import statistics
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# loading and cleaning ENSG converter
test = []
geneId_geneName = {}
with open('Homo_sapiens.GRCh37.74.gtf', 'r') as file:
    for line in file:
        line = line.strip()
        data = line.split('\t')[-1]
        test.append(data)
        if 'gene_name' in data:
            attributes = data.split(';')
            geneId = attributes[0].split(' ')[1].strip('"')
            for attr in attributes:
                if 'gene_name' in attr:
                    geneName = attr.split(' ')[2].strip('"')
                    if geneId not in geneId_geneName:
                        geneId_geneName[geneId] = geneName
geneName_geneId = {v: k for k, v in geneId_geneName.items()}

In [3]:
def createRegulatory(regulatory_filepath):
    # loading, cleaning, and permutating regulatory dataset
    print(f'Loading: {regulatory_filepath}')
    regulatory = pd.read_csv(regulatory_filepath, index_col=0)
    
    exceptions = []
    for name in regulatory.index:
        try:
            regulatory = regulatory.rename(index={name: geneName_geneId[name]})
        except:
            exceptions.append(name)
    
    print(f'Row exception count: {len(exceptions)}')
    
    for exc in exceptions:
        regulatory = regulatory.drop(exc)

    # if columns are ENSG IDs, uncomment this part
    # exceptions = []
    # for ID in regulatory.columns.tolist():
    #     if ID not in geneId_geneName:
    #         exceptions.append(ID)

    # if columns are gene names, uncomment this part
    exceptions = []
    for name in regulatory.columns.tolist():
        try:
            regulatory = regulatory.rename(columns={name: geneName_geneId[name]})
        except:
            exceptions.append(name)

    
    print(f'Column exception count: {len(exceptions)}')

    for exc in exceptions:
        regulatory = regulatory.drop(exc)

    
    def inverse(x):
        return 1/x
    
    def absolute(x):
        return abs(x)
    
    regulatory = regulatory.map(inverse)
    regulatory = regulatory.map(absolute)

    return regulatory

def createEndpoints(filepath):
    # loading and cleaning dataset
    endpoints = pd.read_csv(filepath, sep='\t', index_col=0)
    
    exceptions = []
    for name in endpoints.index:
        try:
            endpoints = endpoints.rename(index={name: geneName_geneId[name.upper()]})
        except:
            exceptions.append(name)
    
    print(f'Endpoints exception count: {len(exceptions)}')
    
    for exc in exceptions:
        endpoints = endpoints.drop(exc)

    return list(endpoints.index)

def createRandomEndpoints(regulatory, num, seed):
    rand = regulatory.sample(n=num, random_state=seed)
    return list(rand.index)

def createGraph(regulatory, endpoints):
    # creating network
    regMatrix = regulatory.to_numpy().tolist()
    
    G = nx.Graph()
    nodes = list(set(list(regulatory.index) + regulatory.columns.tolist() + endpoints))
    G.add_nodes_from(nodes)
    edgeCount = 0
    for rowName, row in zip(regulatory.index, regMatrix):
        for columnName, cell in zip(regulatory.columns.tolist(), row):
            # if cell < 10:
            G.add_edge(rowName, columnName, weight=cell)
            edgeCount += 1
    
    print(f'EdgeCount: {edgeCount}')

    return G

def connectionEnrichment(origins, endpoints):
    datasets = os.listdir('data')
    count = {}
    for i, dataset in enumerate(datasets):
        print(i)
        regulatory = createRegulatory(f'data/{dataset}')
        G = createGraph(regulatory, endpoints)
        for origin in origins:
            for i, endpoint in enumerate(tqdm(endpoints)):
                try:
                    path = nx.shortest_path(G, origin, endpoint, weight="weight")
                    path.pop(0)
                    path.pop()
                    for node in path:
                        if node not in count:
                            count[node] = 1
                        else:
                            count[node] += 1
                except:
                    pass
    return count

def sortDic(dic):
    return dict(reversed(sorted(dic.items(), key=lambda item: item[1])))

def printResultsWithStats(dic):
    print(f'Size: {len(dic)}')
    print(f'Average: {statistics.mean(dic.values())}')
    print(f'Median: {statistics.median(dic.values())}')
    print(dic)

In [4]:
# defining origins
OCT4 = 'ENSG00000229094'
SOX2 = 'ENSG00000181449'
KLF4 = 'ENSG00000136826'
OSKgenes = [OCT4, SOX2, KLF4]

In [5]:
# multi-dataset aging gene enrichment
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
agingCount = connectionEnrichment(OSKgenes, globalAgingGenes)

Endpoints exception count: 15
0
Loading: data/Testis.csv
Row exception count: 0
Column exception count: 30243


KeyError: "['ENSG00000000003'] not found in axis"

In [ ]:
# printing results
printResultsWithStats(aging)

In [ ]:
# control experiment 1
datasets = os.listdir('data')
regulatory = createRegulatory(f'data/{datasets[0]}')
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
randomGenes = createRandomEndpoints(regulatory, len(globalAgingGenes) + len(OSKgenes), 42)
origins = randomGenes[len(globalAgingGenes):]
endpoints = randomGenes[:len(globalAgingGenes)]
controlCount1 = connectionEnrichment(origins, endpoints)

In [ ]:
# printing results
printResultsWithStats(control1)

In [ ]:
# control experiment 2
datasets = os.listdir('data')
regulatory = createRegulatory(f'data/{datasets[0]}')
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
randomGenes = createRandomEndpoints(regulatory, len(globalAgingGenes) + len(OSKgenes), 43)
origins = randomGenes[len(globalAgingGenes):]
endpoints = randomGenes[:len(globalAgingGenes)]
controlCount2 = connectionEnrichment(origins, endpoints)

In [ ]:
# printing results
printResultsWithStats(control2)

In [ ]:
# control experiment 3
datasets = os.listdir('data')
regulatory = createRegulatory(f'data/{datasets[0]}')
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
randomGenes = createRandomEndpoints(regulatory, len(globalAgingGenes) + len(OSKgenes), 44)
origins = randomGenes[len(globalAgingGenes):]
endpoints = randomGenes[:len(globalAgingGenes)]
controlCount3 = connectionEnrichment(origins, endpoints)

In [ ]:
# printing results
printResultsWithStats(control3)

In [ ]:
# creating graph
agingVal = list(aging.values())
controlVal1 = list(control1.values())
controlVal2 = list(control2.values())
controlVal3 = list(control3.values())
agingName = ['aging' for i in range(len(agingVal))]
controlName1 = ['control1' for i in range(len(controlVal1))]
controlName2 = ['control2' for i in range(len(controlVal2))]
controlName3 = ['control3' for i in range(len(controlVal3))]

counts = agingVal + controlVal1 + controlVal2 + controlVal3
names = agingName + controlName1 + controlName2 + controlName3

df = pd.DataFrame({'counts': counts, 'names': names})

sns.set_theme(style="ticks")

f, ax = plt.subplots(figsize=(7, 5))
sns.despine(f)

sns.histplot(
    df,
    x="counts", hue="names",
    multiple="dodge",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    bins = 16,
)

plt.yscale('log')
plt.xlabel("Number of overlaps")

In [ ]:
for name1, dataset1 in zip(['aging', 'control1', 'control2', 'control3'], [aging, control1, control2, control3]):
    print(f'Origin: {name1}')
    for name2, dataset2 in zip(['aging', 'control1', 'control2', 'control3'], [aging, control1, control2, control3]):
        print(f'Endpoint: {name2}')
        intersection = set(dataset1).intersection(set(dataset2))
        sideTotal2 = sum([dataset2[i] for i in intersection])
        print(len(intersection))
        print(sideTotal2)
        print(int(sideTotal2 / len(intersection)))
        print(f'{len(intersection)} // {sideTotal2} // {int(sideTotal2 / len(intersection))}')